<font color='green' size='6'>**Практическая работа 2**</font>  
<font color='#11a642' size='5'>**Тема: Машинное обучение**</font>  
<font size='5'>**Задание:**</font>  

Домашняя работа основана на данных конкурса https://www.kaggle.com/competitions/playground-series-s4e10/

**Ваша задача** — предсказать, останется ли клиент с аккаунтом или закроет его (например, уйдёт).

Требования к оформлению:
- Работа должна быть оформлена в jupyter notebook (файл *.ipynb)
- Работа должна содержать комментарии и выводы. Объясняйте почему вы выбрали именно этот способ обработки данных


Всем удачи!





# <font color='#11a642' size='6'> **Импорт и установка библиотек**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.impute import SimpleImputer

# <font color='#11a642' size='6'> **Загрузка данных**

In [ ]:
# 1. Загрузка датасета
!gdown 1wvqGslWLauKyZxLY-d3RQCTyHCb5wJCO

In [ ]:
try:
  df = pd.read_csv('/train.csv')
except:
  df = pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
# 2. Разделение признаков и целевой переменной
X = df.drop('Exited', axis=1)
y = df['Exited']

In [ ]:
y.head()

## <font color='#11a642' size='5'> Напишите ответы на следующие вопросы: </font>


1.   К какому типу переменных относится целевой признак?
2.   Какую задачу машинного обучения вам предстоит решить, регрессия или классификация?

# ваш ответ

# <font color='#11a642' size='6'> **Разведочный анализ данных**

## <font color='#11a642' size='5'> Оцените размер вашего датасета  </font>



1.   Сколько ГБ ваш датасет
2.   Сколько строк и столбцов



In [ ]:
# Сколько ГБ ваш датасет
# ваш код
dataset_size_gb = df.memory_usage(deep=True).sum() / (1024**3)
print(f"Размер датасета: {dataset_size_gb:.4f} ГБ")


In [ ]:
# Сколько строк и столбцов
# ваш код
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")

## <font color='#11a642' size='5'> Анализ числовых (непрерывных) признаков


- для каждого признака посчитайте минимум, максимум, среднее, медиану

In [ ]:
# ваш код
# Описательные статистики для числовых признаков
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop('Exited')
df[numeric_cols].describe().T

## <font color='#11a642' size='5'> Анализ категориальных признаков


- для каждого признака посчитайте моду, количество уникальных значений

In [ ]:
# ваш код
# Анализ категориальных признаков
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\nПризнак: {col}")
    print(f"Количество уникальных значений: {df[col].nunique()}")
    print(f"Мода: {df[col].mode()[0]}")
    print(df[col].value_counts())

## <font color='#11a642' size='5'> Анализ целевого признака


- какая доля 1?

In [ ]:
# ваш код
# Доля 1 в целевом признаке
print(f"Доля клиентов, которые ушли: {y.mean():.2%}")

# Визуализация распределения
plt.figure(figsize=(6, 4))
sns.countplot(x=y)
plt.title('Распределение целевой переменной')
plt.show()

# <font color='#11a642' size='6'> **Разделите данные на трейн и тест**

- нужна ли стратификация?

In [ ]:
# ваш код
# Разделение данных с стратификацией, так как классы несбалансированы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# <font color='#11a642' size='6'> **Подготовка данных к обучению**

## Поиск и анализ выбросов

In [ ]:
# ваш код
# Визуализация выбросов для числовых признаков
# Фиксируем 3 колонки
n_cols = 3  
# Округление вверх
n_rows = len(numeric_cols) // n_cols + (1 if len(numeric_cols) % n_cols != 0 else 0)

# Создание фигуры с динамическим размером
plt.figure(figsize=(15, 5*n_rows))

# Построение boxplot для каждого числового признака
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot: {col}')
    
plt.tight_layout()
plt.show()

## Анализ и обработка пропусков

In [ ]:
# ваш код
# Проверка на пропуски
# Пропусков нет - обработка не требуется
print("Количество пропусков по столбцам:")
print(df.isnull().sum())

## Анализ и обработка категориальных переменных

In [ ]:
# ваш код
# Создание пайплайна для обработки данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())  # Масштабирование числовых признаков
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

## Масштабирование числовых признаков

In [ ]:
# ваш код
# Масшитабирование числовых признаков происходит в пайплайне

# <font color='#11a642' size='6'> **Построение моделей машинного обучения**

## <font color='#11a642' size='5'> Построение логистической регрессии

In [ ]:
# ваш код
# Создание и обучение модели
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_preprocessed, y_train)

# Предсказания
y_pred = logreg.predict(X_test_preprocessed)
y_pred_proba = logreg.predict_proba(X_test_preprocessed)[:, 1]



## <font color='#11a642' size='5'> Построение логистической регрессии с регуляризацией
> можно использовать [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), настроив гиперпараметры penalty и C

In [ ]:
# ваш код
# Создание и обучение модели с регуляризацией
logreg_l2 = LogisticRegression(penalty='l2', C=0.1, random_state=42, max_iter=1000)
logreg_l2.fit(X_train_preprocessed, y_train)

# Предсказания
y_pred_l2 = logreg_l2.predict(X_test_preprocessed)
y_pred_proba_l2 = logreg_l2.predict_proba(X_test_preprocessed)[:, 1]

# Создание и обучение модели с L1-регуляризацией
logreg_l1 = LogisticRegression(penalty='l1', C=0.1, solver='liblinear', random_state=42, max_iter=1000)
logreg_l1.fit(X_train_preprocessed, y_train)

# Предсказания
y_pred_l1 = logreg_l1.predict(X_test_preprocessed)
y_pred_proba_l1 = logreg_l1.predict_proba(X_test_preprocessed)[:, 1]



### <font color='#11a642' size='4'> Оценка качества  всех моделей


In [ ]:
# ваш код
# Оценка качества
print("Логистическая регрессия без регуляризации:")
print(classification_report(y_test, y_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

print("\nЛогистическая регрессия с L2-регуляризацией:")
print(classification_report(y_test, y_pred_l2))
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_l2):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred_l2):.4f}")

print("\nЛогистическая регрессия с L1-регуляризацией:")
print(classification_report(y_test, y_pred_l1))
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_l1):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred_l1):.4f}")

# <font color='#11a642' size='4'> Ваши выводы
- какую из моделей вы рекомендуете использовать для прогноза? Почему?



ВАШ ОТВЕТ

** Я рекомендую использовать логистическую регрессию с L2-регуляризацией, потому что: **
- Даёт наилучшее качество прогноза (ROC-AUC 0.76).
- Устойчива к переобучению.
- Проста в интерпретации (что важно для бизнес-аналитики).